In [7]:
import pandas as pd
import seaborn as sb
import numpy as np
from sklearn.metrics import confusion_matrix
DS_LABELS = ["AnnualCrop", "Forest", "HerbaceousVegetation", "Highway","Industrial","Pasture","PermanentCrop",
             "Residential", "River", "SeaLake"]

In [11]:
df_pred = pd.read_csv("finished\ResNet50\\pred_test.csv")

# df_greatest = df_pred.to_numpy()
# for i in range(len(df_greatest)):
#     temp = np.zeros(10)
#     temp[df_greatest[i].argmax()] = 1
#     df_greatest[i] = temp
# df_greatest = pd.DataFrame(df_greatest,columns=DS_LABELS)
# print(df_greatest.head())
df_true = pd.read_csv("labels\\Eurosat_idx_test.csv")
conf = confusion_matrix(df_true,df_pred)
sb.heatmap(conf, cmap="Blues", annot=True)

0    HerbaceousVegetation
1                 SeaLake
2                 Highway
3                 SeaLake
4                 SeaLake
dtype: object


ValueError: Classification metrics can't handle a mix of multiclass and continuous-multioutput targets